In [1]:
import python.image_creator_v4 as ic
ic.test()

import numpy as np
import cv2
from os import path, listdir

In image creator version 4. 2020 Feb 3.


In [2]:
p1Loc = path.abspath('../a.000')
p2Loc = path.abspath('../a.101')
print(path.isfile(p1Loc))
print(path.isfile(p2Loc))

pLoc = path.abspath('../a_param.txt')
print( path.isfile( pLoc ))

imgLoc = '../a_model.png'

True
True
True


In [3]:
ic.from_wrapper( pLoc, p1Loc, p2Loc, imgLoc )

Reading particle file: /nfshome/mbo2d/galaxyJSPAM/a.000
Reading particle file: /nfshome/mbo2d/galaxyJSPAM/a.101


In [4]:

print( path.isfile( imgLoc ))
img = cv2.imread(imgLoc)

def showImg( imgLoc = None, img=None ):
    if img == None:
        img = cv2.imread(imgLoc)
    cv2.namedWindow('test', cv2.WINDOW_NORMAL)
    cv2.resizeWindow('test',500,500)
    cv2.imshow('test', img)
    cv2.waitKey(0)
    cv2.destroyAllWindows() #close the image window

True


'\nprint("Checking files")\nprint("\t%s - %s" % ( str( path.isfile(p1Loc)), p1Loc ) )\nprint("\t%s - %s" % ( str( path.isfile(p2Loc)), p2Loc ) )\nprint("\t%s - %s" % ( str( path.isfile(paramLoc)), paramLoc ) )\nprint("\t%s - %s" % ( str( path.isfile(imgLoc)), imgLoc ) )\n\nimgParam = ic.imageParameterClass_v3( paramLoc )\n#print( imgParam.printVal() )\n\npts = readParticleFiles()\npts = readParticleFiles( pts1Loc=p1Loc, pts2Loc=p2Loc )\n\nif pts==None:\n    print("Under construction")\n'

In [7]:
# Find angle between from and to galaxy center vectors
def getRotationAngle( v1, v2 ):    
    
    if v1.shape != (2,) or v2.shape != (2,):
        print("WARNING. Image Creator.getRotationAngle()")
        print("Was not given 2D arrays, returning 0")
        return None   
    
    # signed angle between v1 and (x=1,y=0)
    v1_angle = np.arctan2( v1[1], v1[0] ) # (y,x)
    
    # signed angle between v2 and (x=1,y=0)
    v2_angle = np.arctan2( v2[1], v2[0] ) # (y,x)
    
    # signed angle from v1 to v2
    angle = v2_angle - v1_angle
    
    return angle

In [8]:
def ImgCartConv( x, y, mat_height ):
    
    return np.array( [x, mat_height - y] )

height = 10

x_plot = np.linspace( 0, 10, 5)
y_plot = np.linspace( 0, 10, 5)

for x,y in zip(x_plot,y_plot):
    print(x,y, ImgCartConv(x,y,height))

0.0 0.0 [ 0. 10.]
2.5 2.5 [2.5 7.5]
5.0 5.0 [5. 5.]
7.5 7.5 [7.5 2.5]
10.0 10.0 [10.  0.]


In [45]:
# This is to shift the points so the galaxies match the desired points
# inputs ( galaxy_1_particles, galaxy_2_particles, galaxy_centers, to_image_centers)

def shiftPoints_v3( g1pts, g2pts, ptsCenters, imgParam, printTR=False ):
    
    imgCenters = imgParam.gCenter
    imgHeight = imgParam.nRow
    
    # x,y Cartesian coordinates for galaxy centers of particles
    g1_fc = ptsCenters[0,0:2]
    g2_fc = ptsCenters[1,0:2]
    
    # galaxy centers in pts format
    gc = np.ones((3,2))
    gc[0:2,0] = g1_fc
    gc[0:2,1] = g2_fc  

    # x, y cartesian coordinates for galaxy centers on img
    g1_tc = ImgCartConv( imgCenters[0,0], imgCenters[1,0], imgHeight)
    g2_tc = ImgCartConv( imgCenters[0,1], imgCenters[1,1], imgHeight)
      
    
    # Vector for pts centers if at origin 
    fv = g2_fc - g1_fc
    
    # vector for img center if at origin
    tv = g2_tc - g1_tc
    
    # Find rotation angle between vectors created by galaxy centers.
    theta = getRotationAngle( fv, tv )
        
    # Create Scaling Matrix    
    fl = np.sqrt( fv[0]**2 + fv[1]**2 ) # from center length
    tl = np.sqrt( tv[0]**2 + tv[1]**2 ) # to center length
    scale = tl/fl    
    
    scaleMat = np.eye(3)
    scaleMat[0,0] = scale
    scaleMat[1,1] = scale
    
    # Create Rotation Matrix
    rotMat = np.eye((3))
    rotMat[0,0:2] = [ np.cos( theta ) , -np.sin( theta ) ]
    rotMat[1,0:2] = [ np.sin( theta ) ,  np.cos( theta ) ]
    
    # Create 1st Translation Matrix
    t1Mat = np.eye(3)
    t1Mat[0,2] = g1_tc[0]
    t1Mat[1,2] = g1_tc[1]
    
    
    # Create Matrix to convert pts from cartesian coordinates 
    # to image coord. by reflecting across x-axis and adding imgHeight
    # Equivalent to y_img = imgHeight - y_cart
    
    # reflection matrix across x=0
    reflectMat = np.eye(3)
    reflectMat[1,1] *= -1
    
    # Create 2nd translation matrix to move up imgHeight
    t2Mat = np.eye(3)
    t2Mat[1,2] = imgHeight
    
    # Create single matrix to apply to all points at once
    #   for computational effeciency    
    
    # Rotate points to match image orientation
    adjMat = np.copy(rotMat)

    # Scale points to match image scale
    adjMat = np.matmul( scaleMat, adjMat )

    # reflect points across x-axis for image coord
    #adjMat = np.matmul( reflectMat, adjMat )

    # translate points to desired centers in cartesian coord
    adjMat = np.matmul( t1Mat, adjMat )

    # translate points to image coord
    #adjMat = np.matmul( t2Mat, adjMat )

    

    if printTR:

        print('FROM:')
        print(gc[0:2,:])

        print("To:")
        print(imgCenters)  

        # Rotate points to match image
        print("\nRotate")
        adjMat2 = np.copy(rotMat)
        tMat = np.matmul( rotMat, gc )
        print( np.matmul( adjMat, gc ) )
        print( tMat )  

        # Scale points to match image
        adjMat2 = np.matmul( scaleMat, adjMat )
        print("\nScale")
        tMat = np.matmul( scaleMat, tMat )
        print( np.matmul( adjMat, gc ) )
        print( tMat )

        # translate points to desired centers in cartesian
        adjMat2 = np.matmul( t1Mat, adjMat )    
        print("\nTranslate to final centers cart")
        tMat = np.matmul( t1Mat, tMat )
        print( np.matmul( adjMat, gc ) )
        

        '''
        # reflect points across x-axis
        adjMat2 = np.matmul( reflectMat, adjMat ) 
        print("\nReflect across x-axis")
        tMat = np.matmul( reflectMat, tMat )
        print( np.matmul( adjMat, gc ) )
        print( tMat )  

        # translate points to image coordinates
        adjMat2 = np.matmul( t2Mat, adjMat ) 
        print("\nTranslate up to image coord")
        tMat = np.matmul( t2Mat, tMat )
        print( np.matmul( adjMat, gc ) )
        '''

        print( tMat )
        print("\nShould match")
        print( imgCenters )

        
    # APPLYING MATRICES TO POINTS
    # Transpose points because I  (Matthew Ogden)
    # view lists of points as vertical, not horizontal
    
    
    gc = np.transpose( np.matmul(adjMat, gc) )
    
    n = g1pts.shape[0]
    
    tPts = np.ones((3,n))
    
    tPts[0:2,:] = g1pts[:,0:2].T    
    g1pts[:,0:2] = np.matmul( adjMat, tPts )[0:2,:].T
    
    tPts[0:2,:] = g2pts[:,0:2].T    
    g2pts[:,0:2] = np.matmul( adjMat, tPts )[0:2,:].T
    
    
    #g1pts[:,0:3] = np.transpose(np.matmul(adjMat, np.transpose(g1pts[:,0:3])))
    #g2pts[:,0:3] = np.transpose(np.matmul(adjMat, np.transpose(g2pts[:,0:3])))
    
    
    return g1pts, g2pts, gc



'\ntPoints = np.ones((10,3))\n\nfor i in range(10):\n    tPoints[i,0:2] = np.array([i,i])\n\ntp2 = np.copy(tPoints)\n\n\n\n#print(pts.fCenters)\ntCenters = np.ones((2,3))\ntCenters[1,:] = np.array( [ 5, 5, 0])\nprint(tCenters)\n\nrPts1, rPts2, rCenters = shiftPoints_v3(         tPoints, tp2, tCenters, imgParam, printTR=True )\n\nprint(rPts1)\n\n#print(adjMat)\n#print( np.dot( adjMat, np.transpose( tp2 ) ) )\n#g1pts[:,0:3] = np.transpose(np.matmul(adjMat, np.transpose(g1pts[:,0:3])))\n#   \n\n'

In [46]:
newCreateImage( pLoc, p1Loc, p2Loc, 'here.png')

Reading particle file: /nfshome/mbo2d/galaxyJSPAM/a.000
Reading particle file: /nfshome/mbo2d/galaxyJSPAM/a.101
FROM:
[[ 0.        -9.9395293]
 [ 0.        -4.5854109]]
To:
[[513. 309.]
 [514. 608.]]

Rotate
[[513. 309.]
 [510. 416.]
 [  1.   1.]]
[[ 0.         -9.94159482]
 [ 0.         -4.58093095]
 [ 1.          1.        ]]

Scale
[[513. 309.]
 [510. 416.]
 [  1.   1.]]
[[   0. -204.]
 [   0.  -94.]
 [   1.    1.]]

Translate to final centers cart
[[513. 309.]
 [510. 416.]
 [  1.   1.]]
[[513. 309.]
 [510. 416.]
 [  1.   1.]]

Should match
[[513. 309.]
 [514. 608.]]


[[ 0.         0.         0.       ]
 [-9.9395293 -4.5854109  3.2716137]]
[[513. 510.   1.]
 [309. 416.   1.]]
[[513. 309.]
 [514. 608.]]
